## Use decision optimization

### Step 1: Import the library

Run the following code to import the Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains the two modeling packages, Mathematical Programming (docplex.mp) and Constraint Programming (docplex.cp).

In [284]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

If *CPLEX* is not installed, install CPLEX Community edition.

In [285]:
try:
    import cplex
except:
    raise Exception('Please install CPLEX. See https://pypi.org/project/cplex/')

### Step 1.1: Import data from TM1

In [328]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import element_names_from_element_unique_names, build_pandas_dataframe_from_cellset
from creds import tm1_credentials

with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    data = tm1.cubes.cells.execute_view(cube_name='TRK Load Time', view_name='[Python] TRK Load TIme', private=False)
    loadtimes = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    data = tm1.cubes.cells.execute_view(cube_name='TRK Route', view_name='[Python] TRK Route', private=False)
    routes = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    data = tm1.cubes.cells.execute_view(cube_name='TRK Shipment', view_name='[Python] TRK Shipment', private=False)
    shipments = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_TRK Vehicle Type', view_name='[Python] TRK Vehicle Type', private=False)
    trucks = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_TRK Location', view_name='[Python] TRK Location', private=False)
    spokes = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_TRK Hub', view_name='[Python] TRK Hub', private=False)
    hubs = build_pandas_dataframe_from_cellset(data, multiindex=False)
    
    
    

### Step 2: Model the data
In this scenario, the data is simple and is delivered in the json format under the Optimization github.

In [329]:
from collections import namedtuple

In [330]:
_parameters = namedtuple('parameters', ['maxTrucks', 'maxVolume'])
_location = namedtuple('location', ['name'])
_spoke = namedtuple('spoke', ['name', 'minDepTime', 'maxArrTime'])
_truckType = namedtuple('truckType', ['truckType', 'capacity', 'costPerMile', 'milesPerHour'])
_loadTimeInfo = namedtuple('loadTimeInfo', ['hub', 'truckType', 'loadTime'])
_routeInfo = namedtuple('routeInfo', ['spoke', 'hub', 'distance'])
_triple = namedtuple('triple', ['origin', 'hub', 'destination'])
_shipment = namedtuple('shipment', ['origin', 'destination', 'totalVolume'])

In [331]:
import requests
import json
import decimal 
#r = requests.get("https://github.com/vberaudi/utwt/blob/master/trucking.json?raw=true")
#json_data = json.loads(r.text, parse_float=decimal.Decimal )    

In [336]:
optdate = shipments['Time Date'].unique()[0]

In [290]:
json_data = {}

In [291]:
json_data['Parameters'] = {'maxTrucks': 100, 'maxVolume': 5000}

In [292]:
# Reload Hubs
json_data['Hubs'] = [{'name':hub} for hub in hubs['TRK Hub'].values]

In [293]:
# Reload Spokes
spoke_temp = {(spoke[0],spoke[1]):spoke[2] for spoke in spokes.values}
spoke_nodes = set([spoke[0] for spoke in spokes.values])
json_data['Spokes'] = [{'name':spoke, 'minDepTime':spoke_temp[(spoke,'Min Depature Time')], 'maxArrTime':  spoke_temp[(spoke,'Max Arrival Time')]} for spoke in spoke_nodes]

In [294]:
# Reload Trucks
truck_temp = {(truck[0],truck[1]):truck[2] for truck in trucks.values}
truck_types = trucks['TRK Vehicle Type'].unique()
json_data['TruckTypes'] = [{'truckType':truck, 'capacity':truck_temp[(truck,'Capacity')], \
                            'costPerMile': truck_temp[(truck,'Cost Per Mile')], \
                            'milesPerHour': truck_temp[(truck,'MPH')], \
                           } for truck in truck_types]

In [295]:
# Reload Load Times
json_data['LoadTimes'] = [{'hub':lt[0], 'truckType':lt[1], 'loadTime':lt[3]} for lt in loadtimes.values]

In [296]:
# Reload Routes
json_data['Routes'] = [{'spoke':route[1], 'hub':route[0], 'distance':route[3]} for route in routes.values]

In [297]:
# Reload Shipments
json_data['Shipments'] = [{'origin':shipment[1], 'destination':shipment[2], 'totalVolume':shipment[4]} for shipment in shipments.values]

In [298]:
def read_json_tuples(name, my_namedtuple):
    json_fragment = json_data[name]
    length = len(my_namedtuple._fields)
    ret = []
    for t in json_fragment:
        #print t
        ret2 = [0 for i in range(length)]
        for i in  range(length):
            field = my_namedtuple._fields[i]
            ret2[i] = t[field]
        ret.append(my_namedtuple(*tuple(ret2)))
    return ret

def read_json_tuple(name, my_namedtuple):
    json_fragment = json_data[name]
    length = len(my_namedtuple._fields)
    ret = [0 for i in range(length)]
    for i in  range(length):
        field = my_namedtuple._fields[i]
        ret[i] = json_fragment[field]
    return my_namedtuple(*tuple(ret))

In [299]:
json_data

{'Parameters': {'maxTrucks': 100, 'maxVolume': 5000},
 'Hubs': [{'name': 'G'}, {'name': 'H'}],
 'Spokes': [{'name': 'D', 'minDepTime': 340, 'maxArrTime': 900},
  {'name': 'E', 'minDepTime': 420, 'maxArrTime': 800},
  {'name': 'A', 'minDepTime': 360, 'maxArrTime': 1080},
  {'name': 'C', 'minDepTime': 380, 'maxArrTime': 1200},
  {'name': 'F', 'minDepTime': 370, 'maxArrTime': 1070},
  {'name': 'B', 'minDepTime': 400, 'maxArrTime': 1150}],
 'TruckTypes': [{'truckType': 'BigTruck',
   'capacity': 700,
   'costPerMile': 15,
   'milesPerHour': 45},
  {'truckType': 'SmallTruck',
   'capacity': 400,
   'costPerMile': 10,
   'milesPerHour': 55}],
 'LoadTimes': [{'hub': 'G', 'truckType': 'BigTruck', 'loadTime': 55},
  {'hub': 'G', 'truckType': 'SmallTruck', 'loadTime': 30},
  {'hub': 'H', 'truckType': 'BigTruck', 'loadTime': 50},
  {'hub': 'H', 'truckType': 'SmallTruck', 'loadTime': 35}],
 'Routes': [{'spoke': 'A', 'hub': 'G', 'distance': 200},
  {'spoke': 'B', 'hub': 'G', 'distance': 120},
  {'s

In [300]:
parameters = read_json_tuple(name='Parameters', my_namedtuple=_parameters)
hubs = read_json_tuples(name='Hubs', my_namedtuple=_location)
truckTypes = read_json_tuples(name='TruckTypes', my_namedtuple=_truckType)
spokes = read_json_tuples(name='Spokes', my_namedtuple=_spoke)
loadTimes = read_json_tuples(name='LoadTimes', my_namedtuple=_loadTimeInfo)
routes = read_json_tuples(name='Routes', my_namedtuple=_routeInfo)
shipments = read_json_tuples(name='Shipments', my_namedtuple=_shipment)

### Step 3: Prepare the data

Given the number of teams in each division and the number of intradivisional and interdivisional games to be played, you can calculate the total number of teams and the number of weeks in the schedule, assuming every team plays exactly one game per week. 


The season is split into halves, and the number of the intradivisional games that each team must play in the first half of the season is calculated.

In [301]:
maxTrucks = parameters.maxTrucks;
maxVolume = parameters.maxVolume;

hubIds = {h.name for h in hubs}

spokeIds = {s.name for s in spokes}
spoke = {s.name : s for s in spokes}

truckTypeIds = {ttis.truckType for ttis in truckTypes}
truckTypeInfos = {tti.truckType : tti for tti in truckTypes}

loadTime = {(lt.hub , lt.truckType) : lt.loadTime for lt in loadTimes}

# feasible pathes from spokes to spokes via one hub
triples = {_triple(r1.spoke, r1.hub, r2.spoke) for r1 in routes for r2 in routes if (r1 != r2 and r1.hub == r2.hub)}

Some asserts to check the data follows the guidelines.

In [302]:
# Make sure the data is consistent: latest arrive time >= earliest departure time
for s in spokeIds:
    assert spoke[s].maxArrTime > spoke[s].minDepTime, "inconsistent data"

# The following assertion is to make sure that the spoke
# in each route is indeed in the set of Spokes.
for r in routes:
    assert r.spoke in spokeIds, "some route is not in the spokes"

# The following assertion is to make sure that the hub
# in each route are indeed in the set of Hubs.
for r in routes:
    assert r.hub in hubIds, "some route is not in the hubs"

# The following assertion is to make sure that the origin
# of each shipment is indeed in the set of Spokes.
for s in shipments:
    assert s.origin in spokeIds, "origin is not in the set of Spokes"

# The following assertion is to make sure that the destination
# of each shipment is indeed in the set of Spokes.
for s in shipments:
    assert s.destination in spokeIds, "shipment is not in the set of Spokes"

In [303]:
from math import ceil, floor
# the earliest unloading time at a hub for each type of trucks
earliestUnloadingTime = {(r, t) : int(ceil(loadTime[r.hub, t] + spoke[r.spoke].minDepTime + 60 * r.distance / truckTypeInfos[t].milesPerHour)) for t in truckTypeIds for r in routes}
# the latest loading time at a hub for each type of trucks
latestLoadingTime = {(r, t) : int(floor(spoke[r.spoke].maxArrTime - loadTime[r.hub, t] - 60 * r.distance / truckTypeInfos[t].milesPerHour)) for t in truckTypeIds for r in routes}

# Compute possible truck types that can be assigned on a route
# A type of truck can be assigned on a route only if it can make it to the hub and back
#  before the max arrival time at the spoke.
possibleTruckOnRoute =  {(r, t) : 1 if earliestUnloadingTime[r, t] < latestLoadingTime[r, t] else 0 for t in truckTypeIds for r in routes}

### Step 4: Set up the prescriptive model

In [304]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Windows 64bit
* Python version 3.7.6, located at: C:\ProgramData\Anaconda3\python.exe
* docplex is present, version is (2, 13, 184)
* CPLEX library is present, version is 12.10.0.0, located at: C:\ProgramData\Anaconda3\lib\site-packages
* pandas is present, version is 1.0.1


#### Create the DOcplex model
The model contains all the business constraints and defines the objective.

In [305]:
from docplex.mp.model import Model

model = Model("truck")

#### Define the decision variables

In [306]:
truckOnRoute = model.integer_var_matrix(keys1=routes, keys2=truckTypeIds, lb=0, ub=maxTrucks, name="TruckOnRoute")

# This represents the volumes shipped out from each hub
# by each type of trucks on each triple
# The volumes are distinguished by trucktypes because trucks of different types
# arrive at a hub at different times and the timing is used in defining
# the constraints for volume availability for the trucks leaving the hub.
outVolumeThroughHubOnTruck = model.integer_var_matrix(keys1=triples, keys2=truckTypeIds, lb=0, ub=maxVolume, name="OutVolumeThroughHubOnTruck")

# This represents the volume shipped into each hub by each type of trucks on each triple
# It is used in defining timing constraints.
inVolumeThroughHubOnTruck = model.integer_var_matrix(keys1=triples, keys2=truckTypeIds, lb=0, ub=maxVolume, name="InVolumeThroughHubOnTruck")

#### Express the business constraints

##### The number of trucks of each type should be less than "maxTrucks", and if a type of truck is impossible for a route, its number should be zero

In [307]:
for r in routes:
    for t in truckTypeIds:
        model.add_constraint(truckOnRoute[r, t] <= possibleTruckOnRoute[r, t] * maxTrucks)

##### On each route s-h, the total inbound volume carried by trucks of each type should be less than the total capacity of the trucks of this type.

In [308]:
for (s,h,dist) in routes:
    for t in truckTypeIds:
        model.add_constraint(
            model.sum(inVolumeThroughHubOnTruck[(s1, h1, dest), t] for (s1, h1, dest) in triples if s == s1 and h1 == h)
              <= truckOnRoute[(s, h, dist), t] * truckTypeInfos[t].capacity
        )

##### On any triple, the total flows in the hub = the total flows out the hub

In [309]:
for tr in triples:
    model.add_constraint(
        model.sum(inVolumeThroughHubOnTruck[tr, t] for t in truckTypeIds)
        == model.sum(outVolumeThroughHubOnTruck[tr, t] for t in truckTypeIds)
    )

##### The sum of flows between any origin-destination pair via all hubs is equal to the shipment between the o-d pair.

In [310]:
for (o,d,v) in shipments:
    model.add_constraint(model.sum(inVolumeThroughHubOnTruck[(o1, h, d1), t] for t in truckTypeIds for (o1, h, d1) in triples if o1 == o and d1 == d) == v)

##### There must be enough volume for a truck before it leaves a hub. 
In another words, the shipments for a truck must arrive at the hub from all spokes before the truck leaves. 
The constraint can be expressed as the following: For each route s-h and leaving truck of type t:  Cumulated inbound volume arrived before the loading time of the truck >= Cumulated outbound volume upto the loading time of the truck(including the shipments being loaded).

In [311]:
for (s,h,dist) in routes:
    for t in truckTypeIds:
        model.add_constraint(
            # The expression below defines the indices of the trucks unloaded before truck t starts loading.
          model.sum(inVolumeThroughHubOnTruck[(o, h, s), t1]
                    for (o,h0,s0) in triples if h0 == h and s0 == s
                    for t1 in truckTypeIds
                    for (o2,h2,dist1) in routes if h2 == h0 and o2 == o
                    if earliestUnloadingTime[(o, h, dist1), t1] <= latestLoadingTime[(s, h, dist), t])
            >=
              # The expression below defines the indices of the trucks left before truck t starts loading.
          model.sum(outVolumeThroughHubOnTruck[(o, h, s), t2]
                    for (o,h0,s0) in triples if h0 == h and s0 == s
                    for t2 in truckTypeIds
                    for (o2,h2,dist2) in routes if h2 == h0 and o2 == o
                    if latestLoadingTime[(o, h, dist2), t2] <= latestLoadingTime[(s, h, dist), t])
        )

#### Express the objective

In [312]:
totalCost = model.sum(2 * r.distance * truckTypeInfos[t].costPerMile * truckOnRoute[r, t] for r in routes for t in truckTypeIds)

model.minimize(totalCost)

### Solve with Decision Optimization 

You will get the best solution found after n seconds, due to a time limit parameter.


In [313]:
model.print_information()

assert model.solve(), "!!! Solve of the model fails"
model.report()

Model: truck
 - number of variables: 188
   - binary=0, integer=188, continuous=0
 - number of constraints: 132
   - linear=132
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
* model truck solved with objective = 25850


### Step 5: Investigate the solution and then run an example analysis

In [353]:
model.variables

AttributeError: 'Model' object has no attribute 'variables'

In [314]:
#solution object model
_result = namedtuple('result', ['totalCost'])
_nbTrucksOnRouteRes = namedtuple('nbTrucksOnRouteRes', ['spoke', 'hub', 'truckType', 'nbTruck'])
_volumeThroughHubOnTruckRes = namedtuple('volumeThroughHubOnTruckRes', ['origin', 'hub', 'destination', 'truckType', 'quantity'])
_aggregatedReport = namedtuple('aggregatedReport', ['spoke', 'hub', 'truckType', 'quantity'])

In [315]:
# Post processing: result data structures are exported as post-processed tuple or list of tuples
# Solve objective value
import pandas as pd
result = _result(totalCost.solution_value)


nbTrucksOnRouteRes = pd.DataFrame([_nbTrucksOnRouteRes(r.spoke, r.hub, t, int(truckOnRoute[r, t]))
                      for r in routes
                      for t in truckTypeIds
                      if int(truckOnRoute[r, t]) > 0])


# Volume shipped into each hub by each type of trucks and each pair (origin, destination)
inVolumeThroughHubOnTruckRes = pd.DataFrame([_volumeThroughHubOnTruckRes(tr.origin, tr.hub, tr.destination, t, int(inVolumeThroughHubOnTruck[tr, t]))
                                for tr in triples
                                for t in truckTypeIds
                                if int(inVolumeThroughHubOnTruck[tr, t]) > 0])


# Volume shipped from each hub by each type of trucks and each pair (origin, destination)
outVolumeThroughHubOnTruckRes = pd.DataFrame([_volumeThroughHubOnTruckRes(tr.origin, tr.hub, tr.destination, t, int(outVolumeThroughHubOnTruck[tr, t]))
                                 for tr in triples
                                 for t in truckTypeIds
                                 if int(outVolumeThroughHubOnTruck[tr, t]) > 0])

inBoundAggregated = pd.DataFrame([_aggregatedReport(r.spoke, r.hub, t, sum(int(inVolumeThroughHubOnTruck[tr, t])
                                                              for tr in triples if tr.origin == r.spoke and tr.hub == r.hub))
                     for r in routes
                     for t in truckTypeIds
                     if int(truckOnRoute[r, t]) > 0])
outBoundAggregated = pd.DataFrame([_aggregatedReport(r.spoke, r.hub, t, sum(int(outVolumeThroughHubOnTruck[tr, t])
                                                               for tr in triples if tr.destination == r.spoke and tr.hub == r.hub))
                      for r in routes
                      for t in truckTypeIds
                      if int(truckOnRoute[r, t]) > 0])

In [316]:
from IPython.display import display

AttributeError: 'result' object has no attribute 'solution_value'

In [317]:
display(nbTrucksOnRouteRes)

,spoke,hub,truckType,nbTruck
0,D,G,BigTruck,1
1,A,H,BigTruck,2
2,B,H,BigTruck,1
3,B,H,SmallTruck,1
4,C,H,BigTruck,1
5,C,H,SmallTruck,1
6,D,H,SmallTruck,1
7,E,H,BigTruck,1
8,F,H,BigTruck,1
9,F,H,SmallTruck,1


In [318]:
display(inVolumeThroughHubOnTruckRes)

,origin,hub,destination,truckType,quantity
0,C,H,A,BigTruck,143
1,F,H,C,BigTruck,169
2,F,H,B,BigTruck,157
3,A,H,D,BigTruck,350
4,C,H,D,BigTruck,258
5,E,H,D,BigTruck,78
6,B,H,F,SmallTruck,197
7,D,G,A,BigTruck,75
8,F,H,A,BigTruck,201
9,B,H,E,BigTruck,260


In [319]:
display(outVolumeThroughHubOnTruckRes)

,origin,hub,destination,truckType,quantity
0,C,H,A,BigTruck,143
1,F,H,C,SmallTruck,169
2,F,H,B,SmallTruck,157
3,A,H,D,BigTruck,350
4,C,H,D,BigTruck,258
5,E,H,D,BigTruck,78
6,B,H,F,BigTruck,197
7,D,G,A,SmallTruck,75
8,F,H,A,SmallTruck,201
9,B,H,E,BigTruck,263


In [320]:
display(inBoundAggregated)

,spoke,hub,truckType,quantity
0,D,G,BigTruck,493
1,A,H,BigTruck,1345
2,B,H,BigTruck,666
3,B,H,SmallTruck,400
4,C,H,BigTruck,699
5,C,H,SmallTruck,207
6,D,H,SmallTruck,400
7,E,H,BigTruck,685
8,F,H,BigTruck,699
9,F,H,SmallTruck,144


In [321]:
display(outBoundAggregated)

,spoke,hub,truckType,quantity
0,D,G,BigTruck,0
1,A,H,BigTruck,328
2,B,H,BigTruck,712
3,B,H,SmallTruck,157
4,C,H,BigTruck,450
5,C,H,SmallTruck,557
6,D,H,SmallTruck,0
7,E,H,BigTruck,733
8,F,H,BigTruck,603
9,F,H,SmallTruck,262


## Summary


You learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a Constraint Programming model and solve it with IBM Decision Optimization on Cloud.

#### References
* [Decision Optimization CPLEX Modeling for Python documentation](http://ibmdecisionoptimization.github.io/docplex-doc/)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* Need help with DOcplex or to report a bug? Please go [here](https://stackoverflow.com/questions/tagged/docplex).
* Contact us at dofeedback@wwpdl.vnet.ibm.com.


Copyright &copy; 2017-2019 IBM. IPLA licensed Sample Materials.

In [337]:
truckplan_cellset =  {(optdate, plan[1],plan[0],plan[2], 'Number of Trucks'): plan[3] for plan in nbTrucksOnRouteRes.values}

In [341]:
inVolumeThroughHubOnTruckRes.head()

,origin,hub,destination,truckType,quantity
0,C,H,A,BigTruck,143
1,F,H,C,BigTruck,169
2,F,H,B,BigTruck,157
3,A,H,D,BigTruck,350
4,C,H,D,BigTruck,258


In [346]:
in_cellset = {(optdate, inbound[1], inbound[0], inbound[2], inbound[3], 'Inbound Quantity'):inbound[4] \
              for inbound in   \
               inVolumeThroughHubOnTruckRes.values}

In [347]:
out_cellset = {(optdate, inbound[1], inbound[0], inbound[2], inbound[3], 'Outbound Quantity'):inbound[4] \
              for inbound in   \
               outVolumeThroughHubOnTruckRes.values}

In [348]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    tm1.cubes.cells.write_values('TRK Truck Planning', truckplan_cellset, dimensions=['Time Date', 'TRK Hub', 'TRK Location', 'TRK Vehicle Type', 'TRK Truck Planning Measure'])
    tm1.cubes.cells.write_values('TRK Route Schedule', in_cellset, dimensions=['Time Date', 'TRK Hub', 'TRK Location', 'TRK Destination', 'TRK Vehicle Type', 'TRK Route Schedule Measure'])
    tm1.cubes.cells.write_values('TRK Route Schedule', out_cellset, dimensions=['Time Date', 'TRK Hub', 'TRK Location', 'TRK Destination', 'TRK Vehicle Type', 'TRK Route Schedule Measure'])